# Prétraitement des données de tweets

**Auteur :** Mehdi MUNIM

**Date :** 2023-11-26

**Description :**

Ce notebook prétraite les données de tweets en utilisant les fonctions définies dans le fichier `preprocessing.py`.

In [ ]:
#@title 1. Importation des librairies

import pandas as pd
import sys

# Ajouter le chemin du dossier src pour importer les modules
sys.path.append('../src')
import preprocessing

## 2. Chargement des données

In [ ]:
#@title 2.1 Chargement des données

# Charger les données du fichier CSV
df = pd.read_csv('../data/raw/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

# Renommer les colonnes
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']

# Afficher les premières lignes du DataFrame
print("Premières lignes du DataFrame :")
display(df.head())

## 3. Prétraitement des données

In [ ]:
#@title 3.1 Nettoyage du texte

# Appliquer la fonction de nettoyage du module preprocessing
df['text'] = df['text'].apply(preprocessing.clean_tweet)

# Afficher les premières lignes du DataFrame avec les tweets nettoyés
print("Premières lignes du DataFrame avec les tweets nettoyés :")
display(df.head())

In [ ]:
#@title 3.2 Tokenization

from transformers import BertTokenizer

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokeniser les tweets
X = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors="tf")
y = df['sentiment'].map({0: 0, 4: 1})  # Convertir les labels en 0 et 1

In [ ]:
#@title 3.3 Vectorisation

from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split( 1 
    df['text'], df['sentiment'], test_size=0.2, random_state=42
)

# Vectorisation TF-IDF
X_train_tfidf, X_test_tfidf = preprocessing.vectorize_tfidf(X_train, X_test)

# Vectorisation Word2Vec
X_train_w2v, X_test_w2v = preprocessing.vectorize_word2vec(X_train, X_test)

# Vectorisation Doc2Vec
X_train_d2v, X_test_d2v = preprocessing.vectorize_doc2vec(X_train, X_test)

## 4. Sauvegarde des données prétraitées

In [ ]:
#@title 4.1 Sauvegarde des données

import os

# Chemin d'accès au dossier data
data_path = '../data/processed/'

# Créer le dossier s'il n'existe pas
os.makedirs(data_path, exist_ok=True)

# Sauvegarde des données pour chaque méthode
def save_data(X_train, X_test, methode):
    with open(data_path + f'X_train_{methode}.pickle', 'wb') as f:
        pickle.dump(X_train, f)
    with open(data_path + f'X_test_{methode}.pickle', 'wb') as f:
        pickle.dump(X_test, f)

save_data(X_train_tfidf, X_test_tfidf, "tfidf")
save_data(X_train_w2v, X_test_w2v, "word2vec")
save_data(X_train_d2v, X_test_d2v, "doc2vec")

# Sauvegarde de y_train et y_test
with open(data_path + 'y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)

with open(data_path + 'y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)